In [1]:
import pandas as pd
import csv
import json
import numpy as np
import re

refperiod = "23q3"

# Extract the year and quarter using regular expressions
match = re.match(r'(\d{2})q(\d)', refperiod)

if match:
    year = match.group(1)
    quarter = int(match.group(2))
    
    # Calculate the start month
    start_month = str((quarter - 1) * 3 + 1).zfill(2)
    
    # Calculate the end month
    end_month = str(quarter * 3).zfill(2)
    
    # Combine the year and month to get the desired format
    startMonthID = start_month + year
    endMonthID = end_month + year
    
    print("Start Month:", startMonthID)
    print("End Month:", endMonthID)
else:
    print("Invalid refperiod format.")

Start Month: 0723
End Month: 0923


In [2]:
df = pd.read_csv(f'transaction_resi_converted_raw_csv_{refperiod}.csv', converters = {'leaseDate': str, 'noOfBedRoom': str})
df

,areaSqm,leaseDate,propertyType,district,areaSqft,noOfBedRoom,rent,project,street,y,x
0,180-190,0823,Non-landed Properties,23,2000-2100,4,4500,MERAWOODS,HILLVIEW AVENUE,37591.36252,19806.57039
1,180-190,0823,Non-landed Properties,23,2000-2100,4,7000,MERAWOODS,HILLVIEW AVENUE,37591.36252,19806.57039
2,130-140,0823,Non-landed Properties,23,1500-1600,2,3730,MERAWOODS,HILLVIEW AVENUE,37591.36252,19806.57039
3,120-130,0723,Non-landed Properties,23,1300-1400,3,5050,MERAWOODS,HILLVIEW AVENUE,37591.36252,19806.57039
4,120-130,0723,Non-landed Properties,23,1300-1400,3,4300,MERAWOODS,HILLVIEW AVENUE,37591.36252,19806.57039
...,...,...,...,...,...,...,...,...,...,...,...
16947,40-50,0823,Non-landed Properties,8,500-600,2,4000,UPTOWN @ FARRER,PERUMAL ROAD,32796.58779,30477.87296
16948,60-70,0823,Non-landed Properties,8,600-700,3,5600,UPTOWN @ FARRER,PERUMAL ROAD,32796.58779,30477.87296
16949,40-50,0723,Non-landed Properties,8,500-600,2,4000,UPTOWN @ FARRER,PERUMAL ROAD,32796.58779,30477.87296
16950,40-50,0723,Non-landed Properties,8,500-600,2,3800,UPTOWN @ FARRER,PERUMAL ROAD,32796.58779,30477.87296


In [3]:
df = df[df.propertyType.isin(["Non-landed Properties", "Executive Condominium"])]
df = df[df.noOfBedRoom.isin(["1","2","3","4","5"])]
df['areaSqft'] = np.where(df['areaSqft'] == '>3000', '3000-3000', df['areaSqft'])
df[['areaSqft_lower', 'areaSqft_higher']] = df['areaSqft'].str.split('-', expand=True)

#df = pd.concat([df, df['rental_areaSqft'].str.split('-', expand=True)], axis=1)
df = df.astype({"areaSqft_lower":"int","areaSqft_higher":"int"})
df.dtypes
#df
col = df.loc[: , "areaSqft_lower":"areaSqft_higher"]
df['mean_areaSqft'] = col.mean(axis=1)
df

,areaSqm,leaseDate,propertyType,district,areaSqft,noOfBedRoom,rent,project,street,y,x,areaSqft_lower,areaSqft_higher,mean_areaSqft
0,180-190,0823,Non-landed Properties,23,2000-2100,4,4500,MERAWOODS,HILLVIEW AVENUE,37591.36252,19806.57039,2000,2100,2050.0
1,180-190,0823,Non-landed Properties,23,2000-2100,4,7000,MERAWOODS,HILLVIEW AVENUE,37591.36252,19806.57039,2000,2100,2050.0
2,130-140,0823,Non-landed Properties,23,1500-1600,2,3730,MERAWOODS,HILLVIEW AVENUE,37591.36252,19806.57039,1500,1600,1550.0
3,120-130,0723,Non-landed Properties,23,1300-1400,3,5050,MERAWOODS,HILLVIEW AVENUE,37591.36252,19806.57039,1300,1400,1350.0
4,120-130,0723,Non-landed Properties,23,1300-1400,3,4300,MERAWOODS,HILLVIEW AVENUE,37591.36252,19806.57039,1300,1400,1350.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16947,40-50,0823,Non-landed Properties,8,500-600,2,4000,UPTOWN @ FARRER,PERUMAL ROAD,32796.58779,30477.87296,500,600,550.0
16948,60-70,0823,Non-landed Properties,8,600-700,3,5600,UPTOWN @ FARRER,PERUMAL ROAD,32796.58779,30477.87296,600,700,650.0
16949,40-50,0723,Non-landed Properties,8,500-600,2,4000,UPTOWN @ FARRER,PERUMAL ROAD,32796.58779,30477.87296,500,600,550.0
16950,40-50,0723,Non-landed Properties,8,500-600,2,3800,UPTOWN @ FARRER,PERUMAL ROAD,32796.58779,30477.87296,500,600,550.0


In [4]:
df = df.groupby(['project', 'noOfBedRoom']).mean().reset_index()
df
df = df.astype({"mean_areaSqft":"str"})
df.dtypes
df['mean_areaSqft'] = np.where(df['mean_areaSqft'] == '3000.0', '>3000', df['mean_areaSqft'])
df

,project,noOfBedRoom,district,rent,y,x,areaSqft_lower,areaSqft_higher,mean_areaSqft
0,# 1 LOFT,1,14.0,3250.000000,32778.97962,33581.81022,500.000000,600.000000,550.0
1,# 1 SUITES,1,14.0,3685.714286,32734.88809,33353.01700,514.285714,614.285714,564.2857142857143
2,1 CANBERRA,3,27.0,3825.000000,46578.21809,27611.44860,950.000000,1050.000000,1000.0
3,1 KING ALBERT PARK,2,21.0,4875.000000,35409.52156,22090.84370,1125.000000,1225.000000,1175.0
4,1 KING ALBERT PARK,3,21.0,3900.000000,35409.52156,22090.84370,1200.000000,1300.000000,1250.0
...,...,...,...,...,...,...,...,...,...
3197,ZEDGE,1,11.0,3375.000000,33844.09600,29660.41177,450.000000,550.000000,500.0
3198,ZEDGE,2,11.0,4100.000000,33844.09600,29660.41177,733.333333,833.333333,783.3333333333334
3199,ZEDGE,3,11.0,4915.000000,33844.09600,29660.41177,900.000000,1000.000000,950.0
3200,ZENITH,1,10.0,3950.000000,30553.15754,27685.95968,500.000000,600.000000,550.0


In [5]:
df = df.drop(['areaSqft_lower', 'areaSqft_higher'], axis=1)

In [6]:
df.to_csv(f'transactions_resi_project_bedroom_index_{refperiod}.csv', na_rep='N/A', quoting=csv.QUOTE_NONE, index=False)